In [ ]:
#Functions_Calibration.py created 2024-07-24 10:23:36.942221

import sys
sys.path.append(r'C:\Users\ions\Documents\IonControl\project_dir\QuditLab\config\Scripts')

from Functions_Data import *
from Functions_AWG import *
from Functions_Measurement import *
script_functions = (getGlobal, setGlobal, setScan, startScan, stopScan, getAllData, createTrace, closeTrace, plotPoint, scriptIsStopped, setEvaluation)

def run_fast_calibration(script_functions,delta,real_wait_time,f_offset_input=None,f_upper_input=None,ref_pitimes=None):
    #Qudit_ramsey.py created 2024-06-03 17:12:12.796107

    import numpy as np
    from scipy.linalg import expm
    from scipy.optimize import minimize

    import sys
    import time
    import os
    import datetime
    import glob
    import json
    import numpy as np
    import random
    from scipy.optimize import curve_fit
    sys.path.append(r'C:\Users\ions\Documents\IonControl\project_dir\QuditLab\config\Scripts')

    dt_string = datetime.datetime.now().strftime("%Y%m%d_%H%M")
    year = datetime.datetime.now().strftime("%Y")
    month = datetime.datetime.now().strftime("%m")
    day = datetime.datetime.now().strftime("%d")
    pattern = "Z:\\Lab Data\\D52_Calibration_Ba137\\Fast_calibration_ion_control_raw_data\\fast_calibration_*"
    
    Side_band_cooling_reps = 0
    real_wait_time_bool = True
    f_offset_index = [0,2,0]
    f_upper_index = [-1,4,-3]
    pitime_ref_index = [[-2,4,-4],[-2,3,-3],[2,4,2],[2,4,3],[2,4,4]]

    if f_offset_input and f_upper_input:
        [f_offset,f_upper] = [f_offset_input,f_upper_input]
    else:
        [f_offset,f_upper] = Get_1762_Latest_EOM_Freqs([f_offset_index,f_upper_index])

    if ref_pitimes:
        [pitime_n2,pitime_n1,pitime_0,pitime_p1,pitime_p2] = ref_pitimes
    else:
        [pitime_n2,pitime_n1,pitime_0,pitime_p1,pitime_p2] = Get_1762_Latest_PiTimes(pitime_ref_index)
        
    F1PumpTime = 0.5 #us
    F1PumpReps = 50
    InitReps = 0
    fs = 4e9
    threshold = 10
    
    list_of_inits = [[[-1,3,-2],[0,4,-2],[1,4,0],[2,4,2]],
    [[-2,3,-1],[0,3,0],[1,4,0],[2,4,3]],
    [[-2,2,-1],[-1,3,0],[1,3,2],[2,4,2]],
    [[-2,3,-1],[-1,3,0],[0,3,2],[2,4,3]],
    [[-2,2,-1],[-1,3,0],[0,3,2],[1,3,3]]
    ]

    probe_transitions = [[[0,2,0]]]
    for probe_trans in probe_transitions:

        
        s12_level = probe_trans[0][0]
        
        #detuning = 0.005
        periodicity = 100 #us

        wait_times = [periodicity/4,periodicity*3/4]
        # Phase Calculations ===================================================================

        phases = [np.pi/2 + np.pi, 3*np.pi/2 + np.pi]

        #=======================================================================================

        init_trans = list_of_inits[s12_level+2]
        init_freqs_array = list(Get_1762_EOM_Freqs(init_trans,f_offset,f_upper))
        init_times_array = list(Get_1762_PiTimes(init_trans,pitime_n2,pitime_n1,pitime_0,pitime_p1,pitime_p2))
        init_pulse_time = sum(init_times_array) 
        
        set_freq = list(Get_1762_EOM_Freqs(probe_trans,f_offset,f_upper))
        set_freq = [set_freq[0] + delta]
        print('set freq', set_freq)
        
        
       
        
        pi_times= list(Get_1762_PiTimes(probe_trans,pitime_n2,pitime_n1,pitime_0,pitime_p1,pitime_p2))
        times = []
        for i in range(len(pi_times)):
            times.append(2*pi_times[i]*np.arcsin(np.sqrt(1/(len(pi_times)+1-i)))/np.pi)
        
        
        print(times)
        full_freqs = set_freq + [0] + list(np.flip(set_freq))
        print(full_freqs)
        

        F1_Pump_Time = getGlobal('F1_PumpTime')
        if not "%s"%F1_Pump_Time == "%s us"%F1PumpTime:
            setGlobal("F1_PumpTime", F1PumpTime, "us")
        
        Init_reps = getGlobal('InitialisationReps')
        if not "%s"%Init_reps == InitReps:
            setGlobal("InitialisationReps", InitReps, "")
        
        F1Pump_reps = getGlobal('F1_PumpReps')
        if not "%s"%F1Pump_reps == F1PumpReps:
            setGlobal("F1_PumpReps", F1PumpReps, "")
        
        Init_PulseTime = getGlobal('Init_Shelving_Pulse_Time')
        if not "%s"%Init_PulseTime == init_pulse_time:
            setGlobal("Init_Shelving_Pulse_Time", init_pulse_time, "us")
        
        SB_cooling_reps = getGlobal("Sideband_Cooling_Reps")
        if not "%s"%SB_cooling_reps == Side_band_cooling_reps:
            setGlobal("Sideband_Cooling_Reps", Side_band_cooling_reps, "")
        
        os.system('ssh pi@192.168.168.103 python ~/pll-evalboard-synthesizer/src/Control-Programs/att1.py 16 %s'%10)
        os.system('ssh pi@192.168.168.103 python ~/pll-evalboard-synthesizer/src/Control-Programs/att1.py 13 %s'%0)
        
        pulse_program = "Qudit_ramsey_experiment_fast_calibration"
        
        
                
        def findPiTime_withfit_plotPD(f_offset,f_upper,real_wait_time, threshold, pulse_program, script_functions):
            getGlobal, setGlobal, setScan, startScan, stopScan, getAllData, createTrace, closeTrace, plotPoint, scriptIsStopped, setEvaluation = script_functions
            createTrace('Qudit Ramsey, 0 state', 'Scan Data', xLabel=f'Pulse Time (us)')
            wait_time_num = 0
            fluor_ave = []
            pulse_time_list = []
            file_names_list = []
            while wait_time_num<2:
                if scriptIsStopped():
                    break
                
                #init_trans = list_of_inits[s12_level+2]
                #init_freqs_array = list(Get_1762_EOM_Freqs(init_trans,f_offset,f_upper))
                #init_times_array = list(Get_1762_PiTimes(init_trans,pitime_n2,pitime_n1,pitime_0,pitime_p1,pitime_p2))
                #init_pulse_time = sum(init_times_array) 
        
                #set_freq = list(Get_1762_EOM_Freqs(probe_trans,f_offset,f_upper))
                #set_freq = [545.429083,594.7936,622.70014]
                print('set freq', set_freq)
        
       
                #pi_times= list(Get_1762_PiTimes(probe_trans,pitime_n2,pitime_n1,pitime_0,pitime_p1,pitime_p2))
                #times = []
                #for i in range(len(pi_times)):
                #    times.append(2*pi_times[i]*np.arcsin(np.sqrt(1/(len(pi_times)+1-i)))/np.pi)

                full_freqs = set_freq + [0] + list(np.flip(set_freq))
       
                print('1')
        
        #Pulse Times ==============================================================
                T_wait = real_wait_time
                full_times = list(times) + [T_wait] + list(np.flip(times))
        #Phases =================================================================
                full_phases = [0,0,phases[wait_time_num]]
        #==============================================================
                Ramsey_wait_time_dummy = getGlobal('Ramsey_Wait_Time')
                if not "%s"%Ramsey_wait_time_dummy == "%s us"%sum(full_times):
                    setGlobal("Ramsey_Wait_Time", sum(full_times), "us")
        
                half_pi_time_dummy = getGlobal('Shelving_Pulse_Time')
                if not "%s"%half_pi_time_dummy == "%s us"%sum(full_times):
                    setGlobal("Shelving_Pulse_Time", sum(full_times), "us")
                print('2')

        # AWG commands======================================================================================
                eng.SendAWGCommand("RES", nargout=0)
                time.sleep(5)
                eng.SendAWGCommand("TRAC:DEL:ALL",nargout = 0)
                eng.SendAWGCommand("SOUR:SEQ:DEL:ALL",nargout = 0)
                eng.SendAWGCommand("INIT:CONT 0", nargout=0) #0 sets AWG to externally triggered mode
                eng.SendAWGCommand("OUTP:STAT 1", nargout=0)
                eng.SendAWGCommand("SOUR:FUNC:MODE USER",nargout = 0)
        
                eng.SendAWGCommand("ROSC:SOUR EXT", nargout=0)
                time.sleep(0.1)
                eng.SendAWGCommand("ROSC:SOUR INT", nargout=0)
        
                eng.SendAWGCommand("TRAC:DEL:ALL",nargout = 0)
                eng.SendAWGCommand("SOUR:SEQ:DEL:ALL",nargout = 0)
        
                print('3')
        #----------------------------------------------------------------------------
                matlab_init_freqs = matlab.double(init_freqs_array)
                matlab_init_times = matlab.double(init_times_array)
        #----------------------------------------------------------------------------
                print(full_freqs)
                print(full_times)
                matlab_set_freq = matlab.double(full_freqs)
                matlab_probe_pulse_time = matlab.double(full_times)
                matlab_probe_phases = matlab.double(full_phases)
        #----------------------------------------------------------------------------
                power_factor = matlab.double([1])
                power_factor_dbm = matlab.double([1])
                print('4.5')
                seg_num = 1
                eng.Pulse_upload(matlab_init_freqs,matlab_init_times,fs,seg_num,power_factor,nargout = 0)
                if scriptIsStopped():
                    break
                seg_num = seg_num + 1
                print('5')
                eng.Pulse_upload_dummy(fs,seg_num,nargout = 0) # Dummy 3rd sequence
                seg_num = seg_num + 1
                eng.Pulse_upload_with_phases(matlab_set_freq,matlab_probe_pulse_time,matlab_probe_phases,fs,seg_num,nargout = 0)
                if scriptIsStopped():
                    break
                seg_num = seg_num + 1
                eng.Pulse_upload_dummy(fs,seg_num,nargout = 0) # Dummy 3rd sequence
                print('6')
                # Readout pulses
                for idx, freq in enumerate(set_freq):
        
                    matlab_ro_freq = matlab.double([freq])
                    matlab_ro_time = matlab.double([pi_times[idx]])
                    print(seg_num,[freq],[pi_times[idx]])
                    seg_num = seg_num + 1
                    eng.Pulse_upload(matlab_ro_freq,matlab_ro_time,fs,seg_num,power_factor,nargout = 0)
                    if scriptIsStopped():
                        break
                    seg_num = seg_num + 1
                    eng.Pulse_upload_dummy(fs,seg_num,nargout = 0) # Dummy 3rd sequence
        
        #-------------------------------------------------------------------------------------------------------------------------
                print('7')
        
                eng.SendAWGCommand("SOUR:SEQ:DEF 1,1,1,1", nargout=0)
                eng.SendAWGCommand("SOUR:SEQ:DEF 2,2,1,1", nargout=0)
                eng.SendAWGCommand("SOUR:SEQ:DEF 3,3,1,0", nargout=0)
                eng.SendAWGCommand("SOUR:SEQ:DEF 4,4,1,1", nargout=0)
                seq_num = 4
        
                for idx, freq in enumerate(set_freq):
                    seq_num = seq_num + 1
                    string = f"SOUR:SEQ:DEF {seq_num}, {seq_num},1,0"
                    eng.SendAWGCommand(string, nargout=0)
                    seq_num = seq_num + 1
                    string = f"SOUR:SEQ:DEF {seq_num}, {seq_num},1,1"
                    eng.SendAWGCommand(string, nargout=0)
        
                Table_length_dummy =  getGlobal('Table_length')
                if not "%s"%Table_length_dummy == seq_num:
                    setGlobal("Table_length", seq_num, "")
        
                eng.SendAWGCommand("SOUR:FUNC:MODE SEQ",nargout = 0)
        
                eng.SendAWGCommand("OUTP:COUP DC",nargout = 0)
        
                eng.SendAWGCommand("SOUR:VOLT:LEV:AMPL 1.5",nargout = 0)
        
        
                #total_time = sum([piover2_time,pulse_time,piover2_time])+30
                #PulseTime_Dummy = getGlobal('Shelving_Pulse_Time').magnitude
                #if not "%s"%PulseTime_Dummy == "%s us"%total_time:
                #    setGlobal("Shelving_Pulse_Time", total_time, "us")
                #time.sleep(0.2)
        
                setScan(pulse_program)
                startScan(globalOverrides=list(), wait=True)
                stopScan()
        
                matching_files = glob.glob(pattern)
                print(matching_files)
                matching_files = sorted(matching_files, key=lambda t: os.stat(t).st_mtime)
                print(matching_files)
                file_path = matching_files[-1]
                chunks = file_path.split('\\')
                #print(chunks[-1])        
                fname = chunks[-1]
                file_names_list.append(fname)
                print("=========================================")
        
                arrays = []
                with open(file_path, 'r') as file:
                    print(file_path)
                    for line in file:
                        data = json.loads(line)
                        arrays.append(data[0]["0"][0])
                mean_value = np.mean(np.array(arrays)<threshold)
                print("=========================================")
        
                if np.isnan(mean_value):
                    mean_value = 1
                PD = mean_value
                print("PD is", PD)
                fluor_ave.append(PD)
                pulse_time = wait_times[wait_time_num]
                pulse_time_list.append(wait_times[wait_time_num])
        
                wait_time_num = wait_time_num + 1

                plotPoint(pulse_time, PD,'Qudit Ramsey, 0 state', plotStyle=2)
                
            closeTrace('Qudit Ramsey, 0 state')

            def unitary_product_with_ket_zero(pi_time, T,T_wait, delta, t_wait_T):
                omega = np.pi / (pi_time * 1e-6)
                t_pi_over_2 = (pi_time * 1e-6) / 2
                T = T*1e-6
                T_wait_s = T_wait*1e-6
                def U1(omega, t):
                    H = np.array([[0, omega / 2],
                                  [omega / 2, delta]], dtype=complex)
                    U = expm(-1j * H * t)
                    return U

                def U_wait(omega, t):
                    H = np.array([[0, 0],
                                  [0, delta]], dtype=complex)
                    U = expm(-1j * H * t)
                    return U

                def U2(omega, t):
                    H = np.array([[0, omega * np.exp(-1j * 2 * np.pi * t_wait_T) / 2],
                                  [omega * np.exp(1j * 2 * np.pi * t_wait_T) / 2, delta]], dtype=complex)
                    U = expm(-1j * H * t)
                    return U

                U1 = U1(omega, t_pi_over_2)
                U_wait = U_wait(omega, T_wait_s)
                U2 = U2(omega, t_pi_over_2)

                product = U2 @ U_wait @ U1

                ket_zero = np.array([1, 0], dtype=complex)
                result = product @ ket_zero
                state_pop = np.abs(result)**2
                return state_pop

            def call_unitary_product_twice(pi_time, T,T_wait, delta):
                result1 = unitary_product_with_ket_zero(pi_time, T,T_wait, delta, 1/4)
                result2 = unitary_product_with_ket_zero(pi_time, T,T_wait, delta, 3/4)
                return result1[0], result2[0]

            def fitting_function(delta, pi_time, T,T_wait, target_results):
                computed_result1, computed_result2 = call_unitary_product_twice(pi_time, T,T_wait, delta[0])
                target_result1, target_result2 = target_results
                error = (computed_result1 - target_result1)**2 + (computed_result2 - target_result2)**2
                return error
            pi_time = pi_times[0]
            initial_delta_guess = 0
            result1 = fluor_ave[0]
            result2 = fluor_ave[1]
            target_results = (result1 , result2)
            T = periodicity
            result = minimize(fitting_function, initial_delta_guess, args=(pi_time, T,T_wait, target_results),method='Nelder-Mead')
            optimized_delta = result.x[0]/[2*np.pi]
            print(optimized_delta)
            new_set_frequency = [set_freq[0] - optimized_delta*1e-6]
            freq_string = str(round(set_freq[0],5)).replace('.','p')
            combined_data = zip(pulse_time_list,fluor_ave)
            filename = f'Z:\\Lab Data\\D52_Calibration_Ba137\\Fast_calibration_data\\Fast_calibration_updating_{freq_string}_{T_wait}_{dt_string}.txt'
            with open(filename,'w') as file:
                for x,y in combined_data:
                    file.write(f"{x},{y}\n")
                file.write(f"[{set_freq[0]}],[{new_set_frequency[0][0]}]\n")
                file.write(f"{pi_times},[{pitime_n2},{pitime_n1},{pitime_0},{pitime_p1},{pitime_p2}]\n")
                file.write(f"{phases}\n")
                file.write(f"{file_names_list}\n")

            return new_set_frequency[0]
        
        setEvaluation('Eval3')
        
        new_set_frequency = findPiTime_withfit_plotPD(f_offset,f_upper,real_wait_time, threshold, pulse_program, script_functions)
        print(new_set_frequency)
        setEvaluation('Eval2')
        return new_set_frequency
#freq_list, pitime_list = run_calibration(script_functions,
#                                        do_pi_time_calibration=True,
#                                        low_power=False,
#                                        f_offset_input=545.4315614029318,
#                                        f_upper_input=623.232553,
#                                        ref_pitimes=[23.699, 46.327, 50.2767, 37.142, 46.797])
#print(freq_list)
#print(pitime_list)

f_offset_dummy = getGlobal('f_offset')
if not "%s"%f_offset_dummy == "%s MHz"%freq_list[0]:
    setGlobal("f_offset", freq_list[0], "MHz")
    
f_upper_dummy = getGlobal('f_upper')
if not "%s"%f_upper_dummy == "%s MHz"%freq_list[1]:
    setGlobal("f_upper", freq_list[1], "MHz")

pitime_n2_dummy = getGlobal('pitime_n2')
if not "%s"%pitime_n2_dummy == "%s us"%pitime_list[0]:
    setGlobal("pitime_n2", pitime_list[0], "us")

pitime_n1_dummy = getGlobal('pitime_n1')
if not "%s"%pitime_n1_dummy == "%s us"%pitime_list[1]:
    setGlobal("pitime_n1", pitime_list[1], "us")

pitime_0_dummy = getGlobal('pitime_0')
if not "%s"%pitime_0_dummy == "%s us"%pitime_list[2]:
    setGlobal("pitime_0", pitime_list[2], "us")

pitime_p1_dummy = getGlobal('pitime_p1')
if not "%s"%pitime_p1_dummy == "%s us"%pitime_list[3]:
    setGlobal("pitime_p1", pitime_list[3], "us")

pitime_p2_dummy = getGlobal('pitime_p2')
if not "%s"%pitime_p2_dummy == "%s us"%pitime_list[4]:
    setGlobal("pitime_p2", pitime_list[4], "us")

import matlab.engine
eng = matlab.engine.start_matlab()
eng.addpath(r'C:\Users\ions\Documents\MATLAB\iqtools',nargout = 0)
eng.SendAWGCommand("RES", nargout=0)
time.sleep(1)

deltas = np.linspace(-0.001,0.001,200)
f_offset_input1=545.4316956099669
for delta in deltas:
    delta = 0
    new_set_frequency = run_fast_calibration(script_functions, delta, 500,f_offset_input=f_offset_input1, f_upper_input=623.23257,ref_pitimes=[28.206, 54, 54.605, 40.916, 50])
    f_offset_input1=new_set_frequency
print(new_set_frequency)



